In [ ]:
!pip install numpy==1.24.4 umap==0.1.1 umap-learn==0.5.7 pandas==2.2.2 scikit-learn==1.5.2 matplotlib==3.10.0 scipy==1.13.1 gensim==4.3.3

In [ ]:
import umap
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy.spatial.distance import cosine
from scipy.stats import entropy
from collections import defaultdict

Load preprocessed files

In [ ]:
with open('umap_result.pkl', 'rb') as f:
    umap_result = pickle.load(f)

with open('preprocessed_docs.pkl', 'rb') as f:
    preprocessed_docs = pickle.load(f)

with open('lda_model.pkl', 'rb') as f:
    lda_model = pickle.load(f)

with open('bow_corpus.pkl', 'rb') as f:
    bow_corpus = pickle.load(f)

with open('dictionary.dict', 'rb') as f:
    dictionary = pickle.load(f)

In [ ]:
# Load the preprocessed_docs.csv that contains the year of publication data in 'Year' column and convert to numeric
file_name = 'preprocessed_docs.csv'
df = pd.read_csv(file_name, low_memory=False)

# Filter and preprocess the dataset
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df.reset_index(drop=True, inplace=True)

# Replace NaN years with 1 as a placeholder
df['Year'] = df['Year'].fillna(1).astype(int)

print("Length of preprocessed_docs:", len(preprocessed_docs))
print("Length of df:", len(df))

# Load the Leiden labels
leiden_labels_path = 'leiden_labels_all.csv'
leiden_labels = pd.read_csv(leiden_labels_path)

Plot UMAP divided by decades

In [ ]:
# Create a new column for year ranges
def classify_year_range(year):
    if 1975 <= year <= 1984:
        return '1975-1984'
    elif 1985 <= year <= 1994:
        return '1985-1994'
    elif 1995 <= year <= 2004:
        return '1995-2004'
    elif 2005 <= year <= 2014:
        return '2005-2014'
    elif 2015 <= year <= 2023:
        return '2015-2023'
    else:
        return 'Unknown'

df['Year_Range'] = df['Year'].apply(classify_year_range)

# Create a dataframe with umap embeddings and year range
umap_df = pd.DataFrame(umap_result, columns=['UMAP1', 'UMAP2'])
combined_df = pd.concat([umap_df, df[['Year_Range']]], axis=1)

# Plot individual UMAPs for each year range
year_ranges = ['1975-1984', '1985-1994', '1995-2004', '2005-2014', '2015-2023']
plt.figure(figsize=(25, 5))
for i, year_range in enumerate(year_ranges):
    plt.subplot(1, 5, i + 1)
    subset = combined_df[combined_df['Year_Range'] == year_range]
    plt.scatter(subset['UMAP1'], subset['UMAP2'], c='navy', s=0.5, alpha=0.2)
    plt.title(f'({year_range})')
    plt.xticks([])
    plt.yticks([])
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)

plt.tight_layout()
plt.savefig('Year ranges UMAP all.png', dpi=600)
plt.show()

Temporal evolution of cluster proportions

In [ ]:
# Define cluster names
cluster_names = {
    0: 'Dementia', 1: 'Genetics', 2: 'Geriatrics', 3: 'Gender', 4: 'Cell signaling & stem cells',
    5: 'Exercise', 6: 'Diabetes', 7: 'Hormones', 8: 'Nutrition', 9: 'Statistics',
    10: 'Broad aging terminology', 11: 'Depression & psychology', 12: 'Immunology',
    13: 'Brain structure', 14: 'Memory & learning', 15: 'Healthcare', 16: 'Oxidative stress',
    17: 'Animal studies', 18: 'Muscle', 19: 'DNA damage', 20: 'Comparative studies',
    21: "Alzheimer's", 22: 'Vascular', 23: 'Skin', 24: 'Telomeres', 25: 'Cancer',
    26: 'Cell cycle & senescence', 27: 'Mitochondria', 28: 'Sleep', 29: 'Mouse studies',
    30: 'Bone', 31: 'Physics'
}

# Add Leiden labels to the dataframe
df['Cluster'] = leiden_labels['leiden']

# Group by year and cluster to get the count of documents per cluster per year
yearly_counts = df.groupby(['Year', 'Cluster']).size().unstack(fill_value=0)
total_docs_per_year = yearly_counts.sum(axis=1)
proportion_counts = yearly_counts.div(total_docs_per_year, axis=0)
moving_avg = proportion_counts.rolling(window=5, min_periods=1).mean()

# Filter data for the years 1975 to 2023
moving_avg_filtered = moving_avg[(moving_avg.index >= 1975) & (moving_avg.index <= 2023)]

# Calculate total documents per cluster
total_per_cluster = yearly_counts.sum().sort_values(ascending=False)

# Re-order cluster_names based on total document count
sorted_cluster_names = {cluster: cluster_names[cluster] for cluster in total_per_cluster.index if cluster in cluster_names}

# Plot the results in a single PDF
fig, axes = plt.subplots(nrows=6, ncols=6, figsize=(40, 25))
axes = axes.flatten()

for idx, (cluster, name) in enumerate(sorted_cluster_names.items()):
    if cluster in moving_avg_filtered.columns:
        axes[idx].plot(moving_avg_filtered.index, moving_avg_filtered[cluster], linestyle='-', color='navy')
        axes[idx].set_title(f'{name}', fontsize=24)
        axes[idx].set_xlabel('Year', fontsize=24)
        axes[idx].set_ylabel('Proportion of Documents', fontsize=20)
        axes[idx].tick_params(axis='both', which='major', labelsize=20)
        axes[idx].grid(False)
    else:
        axes[idx].axis('off')

# Hide any unused subplots
for ax in axes[len(sorted_cluster_names):]:
    ax.set_visible(False)
plt.tight_layout()
plt.savefig('Cluster_Evolution_Panel.pdf')
plt.show()

Calculate mean publication year per cluster

In [ ]:
# Filter documents from 1975 or newer and add leiden labels
df_filtered = df[df['Year'] >= 1975].copy().reset_index(drop=True)
df_filtered.loc[:, 'Cluster'] = leiden_labels['leiden'].reset_index(drop=True)

# Calculate the mean year for each cluster
mean_year_per_cluster = df_filtered.groupby('Cluster')['Year'].mean().reset_index()
mean_year_per_cluster['Cluster_Name'] = mean_year_per_cluster['Cluster'].map(cluster_names)

# Plot the mean year for each cluster
plt.figure(figsize=(12, 8))
mean_year_per_cluster_sorted = mean_year_per_cluster.sort_values('Year', ascending=False)
sns.barplot(x='Year', y='Cluster_Name', data=mean_year_per_cluster_sorted, palette='RdBu')
plt.xlabel('Mean Year of Publication')
plt.ylabel('Cluster')
plt.title('Mean Year of Publication per Cluster')
plt.xlim(2008, 2012)
plt.xticks(ticks=range(2008, 2013, 2))
plt.savefig('Mean year of publication per cluster.pdf')
plt.show()

Cosine similarity trend

In [ ]:
# Get topic distribution for each document using LDA model
topic_distributions = []
for doc_bow in bow_corpus:
    doc_topics = lda_model.get_document_topics(doc_bow, minimum_probability=0)
    topic_vector = np.zeros(lda_model.num_topics)
    for topic_id, prob in doc_topics:
        topic_vector[topic_id] = prob
    topic_distributions.append(topic_vector)

df_topics = pd.DataFrame(topic_distributions)

# Add cluster labels and years
df['Leiden_Cluster'] = leiden_labels['leiden']
df_topics['Year'] = df['Year']
df_topics['Cluster'] = df['Leiden_Cluster']

# Function to calculate cosine similarity
def cosine_similarity(v1, v2):
    return 1 - cosine(v1, v2)

# Analyze topic distributions over time within each cluster
results = defaultdict(list)
years = sorted(df['Year'].dropna().unique())

for cluster in df['Leiden_Cluster'].unique():
    cluster_data = df_topics[df_topics['Cluster'] == cluster]
    for i in range(len(years) - 1):
        year_1 = years[i]
        year_2 = years[i + 1]

        # Get average topic distribution for both years
        dist_year_1 = cluster_data[cluster_data['Year'] == year_1].mean(axis=0).values[:-2]
        dist_year_2 = cluster_data[cluster_data['Year'] == year_2].mean(axis=0).values[:-2]

        # Compute cosine similarity between distributions
        cosine_sim = cosine_similarity(dist_year_1, dist_year_2)
        results[cluster].append({
            'Year_1': year_1,
            'Year_2': year_2,
            'Cosine_Similarity': cosine_sim,
        })

# Convert results to a dataframe
df_results = pd.DataFrame([
    {'Cluster': cluster, 'Year_1': res['Year_1'], 'Year_2': res['Year_2'],
     'Cosine_Similarity': res['Cosine_Similarity']}
    for cluster, result in results.items() for res in result
])

In [ ]:
# Filter data from the year 1975 onwards
df_results_filtered = df_results[df_results['Year_1'] >= 1975].copy()
unique_clusters = sorted(df_results_filtered['Cluster'].unique())

# Prepare a dictionary to store trends
cluster_trends = {}

# Iterate over each cluster to calculate the trend
for cluster in unique_clusters:
    cluster_data = df_results_filtered[df_results_filtered['Cluster'] == cluster].copy()
    # Compute the 5-year moving average
    cluster_data['Cosine_MA'] = cluster_data['Cosine_Similarity'].rolling(window=5).mean()
    cluster_data = cluster_data.dropna(subset=['Cosine_MA'])

    # Fit a linear regression model to find the slope
    X = cluster_data['Year_1'].values.reshape(-1, 1)
    y = cluster_data['Cosine_MA'].values
    reg = LinearRegression().fit(X, y)
    slope = reg.coef_[0]
    cluster_trends[cluster_names[cluster]] = slope

# Bar plot of cosine similarity trends
sorted_trends = sorted(cluster_trends.items(), key=lambda x: x[1])
cluster_names_sorted = [item[0] for item in sorted_trends]
slopes_sorted = [item[1] for item in sorted_trends]
plt.figure(figsize=(10, 8))
plt.barh(cluster_names_sorted, slopes_sorted, color='navy')
plt.xlabel('Trend (Slope of 5-Year Moving Average)')
plt.ylabel('Cluster')
plt.title('Trend of Cosine Similarity Across Clusters')
plt.savefig('Trend of Cosine Similarity Across Clusters.pdf', bbox_inches='tight')
plt.show()